In [1]:
import re
import spacy
from spaczz.matcher import FuzzyMatcher
import sentence_transformers
import pandas as pd
import skweak
import random
import sys
import random
import time
from preprocess import preProcessor, refine_string

2023-05-25 13:01:47.148420: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 13:01:54.858918: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-05-25 13:01:54.859169: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-05-25 13:01:54.859181: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# pre-process the addresses
pp = preProcessor('Addresses.csv')

# remove small addresses
cleaned_addr = pp.cleaned_addr_list

# correct the addresses, removing short form
corrected_addr = pp.corrected_addr_list

print(len(corrected_addr), corrected_addr[:10])

/home/lab_covid2/.local/lib/python3.8/site-packages/spaczz/matcher/_phrasematcher.py:242: KwargsWarning: There are more patterns then there are kwargs.
                Patterns not matched to a kwarg dict will have default settings.
  warnings.warn(


6322 ['bt kawade road', 'c2 1104 bel castle society lonkar mala keshav nagar mundhwa', 'koregaon park lane no 5 nr devi mandir kawdewadi', 'fursungi saykarwadi', 'janta vasahat parvati paytha 124 infront pmc school 09', 'kale padal hadapsar railway crossing', 's n 53 1 sanathnagar kharadi', 'g 16 sai ganesh complex karvenagar near icici bank', 'bacchu adda camp', 'lane no 11']


In [3]:
# reading the excel file
sds=pd.read_excel('Localities_list.xlsx')

def get_col(entity, fileObj=sds):
    raw_ents = list(set(fileObj[entity].tolist()))
    ents = [ent.strip() for ent in raw_ents if type(ent) != float]
    ents = [refine_string(ent) for ent in ents]
    return ents

# getting lists of wards, prabhags and localities
wards = get_col('Ward')
prabhags = get_col('Prabhag')
localities = get_col('Localities')

In [4]:
sds

,Localities,Ward,Prabhag,Prabhag_Number,Priority,Alias
0,Dhanori,Yerawada - Kalas - Dhanori,Kalas-Dhanori,1,3,NaN
1,Kalas,Yerawada - Kalas - Dhanori,Kalas-Dhanori,1,3,NaN
2,Gangakunj,Yerawada - Kalas - Dhanori,Kalas-Dhanori,1,0,NaN
3,Lakshmi,Yerawada - Kalas - Dhanori,Kalas-Dhanori,1,0,NaN
4,R&DE,Yerawada - Kalas - Dhanori,Kalas-Dhanori,1,0,NaN
...,...,...,...,...,...,...
844,Uruli,NaN,Uruli Devachi,42,0,NaN
845,Uruli Kanchan,NaN,Uruli Devachi,42,0,NaN
846,Shewalwadi,NaN,Uruli Devachi,42,0,NaN
847,Holkar wadi,NaN,Uruli Devachi,42,0,NaN


In [5]:
# setting localities and prabhags dictionary
localities_dic = {i: set() for i in wards}
prabhag_dic = {i: set() for i in wards}

for index, row in sds.iterrows():
    if type(row['Ward']) != float:
        localities_dic[refine_string(row['Ward'])].add(refine_string(row['Localities']))
        prabhag_dic[refine_string(row['Ward'])].add(refine_string(row['Prabhag']))

for ward in wards:
    localities_dic[ward] = list(localities_dic[ward])
    prabhag_dic[ward] = list(prabhag_dic[ward])

In [6]:
# getting sentence transformer embeddings
model_st = sentence_transformers.SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model_st.encode(corrected_addr)

# similarity matrix
sim_matrix = sentence_transformers.util.cos_sim(embeddings, embeddings)

In [7]:
# similarity vector
sim_matrix[1]

tensor([0.4614, 1.0000, 0.6181,  ..., 0.4745, 0.3452, 0.6241])

In [8]:
# nearest addresses by sentence-transformers embedding
def nearest_tokens_st(query_ind:str, threshhold=0.85 ,addresses = corrected_addr,  sim_matrix=sim_matrix, n=20):
    sim_vec = sim_matrix[query_ind].tolist()
    sim_vec_copy = sim_vec.copy()
    ans = []
    for i in range(n):
        max_el = max(sim_vec)
        if max_el > threshhold:
            max_ind = sim_vec_copy.index(max_el)
            ans.append((max_ind, sim_vec_copy[max_ind], addresses[max_ind]))
            sim_vec.remove(max_el)
        else:
            break 
    return ans
        

In [9]:
#############TEST################
nearest_tokens_st(305)

[(305,
  1.000000238418579,
  'sr no 44 near rajiv gandhi park behind datta mandir kelewadi ko'),
 (115,
  0.9721863865852356,
  'sr no 44 rajiv gandhi park near datta mandir kelewadi ko'),
 (1876,
  0.913855791091919,
  'sr no 44 near datta mandir near shiv prashithan kelewadi ko')]

In [10]:
# defining a matcher object
def entity_matcher(doc, matcher):
    matches = matcher(doc)
    if len(matches) > 0:
        return matches
    else:
        return []

# given a value, finding the key from dic
def find_key(dic, value):
    for k, v in dic.items():
        if value in v:
            return k
    return None

In [11]:
# building the dictionary that matches the address with ward names in it.

nlp = spacy.blank('en')

silver_truth_dic = {i: set() for i in wards}
ward_matcher = FuzzyMatcher(nlp.vocab)
for ward in wards:
    ward_matcher.add(ward, [nlp(ward)])
    
for addr in corrected_addr:
    matches = entity_matcher(nlp(addr), ward_matcher)
    for match in matches:
        if match[-1] > 85:
            silver_truth_dic[match[0]].add(addr)
            



In [12]:
# for matching more addresses splitting the ward names

wards_should_be_splitted_for_fuzzy = []
for i in silver_truth_dic:
    print(i, len(silver_truth_dic[i]))
    if  len(silver_truth_dic[i]) < 10:
        wards_should_be_splitted_for_fuzzy.append(i)

Hadapsar Mundhawa 0
Kasba Vishrambaugwada 0
Kondhawa Yewalewadi 2
Warje Karvenagar 2
Bibwewadi 59
Wanawadi Ramtekadi 0
Yerawada Kalas Dhanori 0
Aundh Baner 0
Kothrud Bavdhan 0
Dhole Patil Road 6
Shivajinagar Ghole Road 0
Sinhgad Road 83
Nagar Road Vadgaon Sheri 8
Dhankawadi Sahakarnagar 0
Bhawani Peth 76


In [13]:
## Updating silver_truth_dic

silver_truth_dic = {i: set() for i in wards}

ward_matcher = FuzzyMatcher(nlp.vocab)
for ward in wards:
    ward_matcher.add(ward, [nlp(ward)])
    if ward in wards_should_be_splitted_for_fuzzy:
        ward_splitted = ward.split(' ')
        ward_matcher.add(ward, [nlp(' '.join(ward_splitted[:int(len(ward_splitted)/2)]).strip())])
        ward_matcher.add(ward, [nlp(' '.join(ward_splitted[int(len(ward_splitted)/2):]).strip())])
    
for addr in corrected_addr:
    matches = entity_matcher(nlp(addr), ward_matcher)
    for match in matches:
        if match[-1] > 85:
            silver_truth_dic[match[0]].add(addr)
            

In [14]:
# wards names and the number of addresses it matches to
for i in silver_truth_dic:
    print(i, len(silver_truth_dic[i]))

Hadapsar Mundhawa 259
Kasba Vishrambaugwada 20
Kondhawa Yewalewadi 223
Warje Karvenagar 376
Bibwewadi 59
Wanawadi Ramtekadi 124
Yerawada Kalas Dhanori 247
Aundh Baner 70
Kothrud Bavdhan 212
Dhole Patil Road 22
Shivajinagar Ghole Road 15
Sinhgad Road 83
Nagar Road Vadgaon Sheri 84
Dhankawadi Sahakarnagar 232
Bhawani Peth 76


In [15]:
# LF 1: heuristic to detect ward name occurrences
def fuzzy_ward_detector(doc):
    matches = entity_matcher(doc, ward_matcher)
    for match in matches:
        if match[-1] > 85:
            yield 0, len(doc), match[0]
            
# LF 2: heuristic to detect prabhag name 
def fuzzy_prabhag_detector(doc):    
    #(checking matching wards)
    matcher = FuzzyMatcher(nlp.vocab)
    for prabhag in prabhags:
        matcher.add(prabhag, [nlp(prabhag)])

    # print(dir(matcher), matcher.name, matcher.patterns, matcher.defaults)
    matches = entity_matcher(doc, matcher)
    for match in matches:
        if match[-1] > 85:
            ward =  find_key(prabhag_dic, match[0])
            if ward != None:
                yield 0, len(doc), ward                
    
# LF 3: heuristic to detect locality name 
def fuzzy_locality_detector(doc):
    matcher = FuzzyMatcher(nlp.vocab)
    for loc in localities:
        matcher.add(loc, [nlp(loc)])

    # print(dir(matcher), matcher.name, matcher.patterns, matcher.defaults)
    matches = entity_matcher(doc, matcher)
    for match in matches:
        if match[-1] > 99:
            ward =  find_key(localities_dic, match[0])
            if ward != None:
                yield match[1], match[2], ward
    

                
# LF 4: first find > 0.92 similarity addresses and look if any of them are in silver truth If it is, label it
def st_similarity_detector(doc):
    doc_ind = corrected_addr.index(doc.text)
    nearest_addr_st = nearest_tokens_st(doc_ind, threshhold=0.9, n=10)
    for addr in nearest_addr_st:
        if addr[2] != doc.text:
            ward = find_key(silver_truth_dic,addr[2])
            if ward != None:
                yield 0, len(doc), ward           
                break

In [16]:
# Building labelling pipeline

lf1 = skweak.heuristics.FunctionAnnotator("fuzzy_ward", fuzzy_ward_detector)
lf2 = skweak.heuristics.FunctionAnnotator("fuzzy_prabhag", fuzzy_prabhag_detector)
lf3 = skweak.heuristics.FunctionAnnotator("fuzzy_locality", fuzzy_locality_detector)
lf4 = skweak.heuristics.FunctionAnnotator("st_ward", st_similarity_detector)

all_annotators = skweak.base.CombinedAnnotator()
all_annotators.add_annotator(lf1)
all_annotators.add_annotator(lf2)
all_annotators.add_annotator(lf3)
all_annotators.add_annotator(lf4)

In [17]:
docs = list(all_annotators.pipe(nlp.pipe(corrected_addr)))

##### Aggregation model
model_label = 'hmm'
model = skweak.aggregation.HMM(model_label, wards, sequence_labelling = False)

# model_lable = 'mv'
# model = skweak.aggregation.MajorityVoter(model_label, wards, sequence_labelling=False)

# fit and annotate train data
model.fit(docs)

# apply model to test data (works as a "predict" function?)
docs = list(model.pipe(docs))


Starting iteration 1
Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Finished E-step with 4616 documents
Starting iteration 2


         1       22938.2324             +nan


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Finished E-step with 4616 documents
Starting iteration 3


         2       22949.0694         +10.8370


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Finished E-step with 4616 documents
Starting iteration 4


         3       22960.1638         +11.0944


Number of processed documents: 1000
Number of processed documents: 2000
Number of processed documents: 3000
Number of processed documents: 4000
Finished E-step with 4616 documents


         4       22970.7748         +10.6110


In [18]:
#################RESULTS####################
aggregators = ['fuzzy_ward','fuzzy_prabhag','st_ward', 'fuzzy_locality']
c = 0
for d in docs:
    print('\n')
    print(d.text)
    if len(d.spans[model_label]) > 0:
        c += 1
        print(d.spans[model_label][0].label_, [model_label])
        labels = []
        for agg in aggregators:   
            if len(d.spans[agg]) > 0:
                labels.append(d.spans[agg][0].label_)
                print(d.spans[agg][0].label_, [agg])  
    else:
        continue

# printing the address and then the label with corresponding labelling function in bracket
print(c, len(corrected_addr))



bt kawade road


c2 1104 bel castle society lonkar mala keshav nagar mundhwa
Hadapsar Mundhawa ['hmm']
Hadapsar Mundhawa ['fuzzy_ward']
Hadapsar Mundhawa ['fuzzy_locality']


koregaon park lane no 5 nr devi mandir kawdewadi
Dhole Patil Road ['hmm']
Dhole Patil Road ['fuzzy_locality']


fursungi saykarwadi


janta vasahat parvati paytha 124 infront pmc school 09
Sinhgad Road ['hmm']
Sinhgad Road ['fuzzy_locality']


kale padal hadapsar railway crossing
Hadapsar Mundhawa ['hmm']
Hadapsar Mundhawa ['fuzzy_ward']
Hadapsar Mundhawa ['fuzzy_locality']


s n 53 1 sanathnagar kharadi
Nagar Road Vadgaon Sheri ['hmm']
Nagar Road Vadgaon Sheri ['fuzzy_locality']


g 16 sai ganesh complex karvenagar near icici bank
Warje Karvenagar ['hmm']
Warje Karvenagar ['fuzzy_ward']
Warje Karvenagar ['fuzzy_prabhag']
Warje Karvenagar ['fuzzy_locality']


bacchu adda camp


lane no 11


sahayog nagar s no 45 warje
Warje Karvenagar ['hmm']
Warje Karvenagar ['fuzzy_ward']
Warje Karvenagar ['st_ward']
Warje Kar

Kothrud Bavdhan ['fuzzy_ward']
Kothrud Bavdhan ['st_ward']
Kothrud Bavdhan ['fuzzy_locality']


vaiduwadi
Wanawadi Ramtekadi ['hmm']
Wanawadi Ramtekadi ['fuzzy_locality']


vyankatesh rivhera lane no 2 sahu colony karvenagar
Warje Karvenagar ['hmm']
Warje Karvenagar ['fuzzy_ward']
Warje Karvenagar ['fuzzy_prabhag']
Warje Karvenagar ['fuzzy_locality']


kalwad vasti lohegaon 411047
Nagar Road Vadgaon Sheri ['hmm']
Nagar Road Vadgaon Sheri ['fuzzy_locality']


santosh nagar near ganpati mandir katraj
Dhankawadi Sahakarnagar ['hmm']
Dhankawadi Sahakarnagar ['fuzzy_locality']


ofw bt kawade road bharat forge company 411007


399 416 b budhwar peth
Bhawani Peth ['hmm']
Bhawani Peth ['fuzzy_locality']


sr no 44 near bal shivaji mitramandal kelewadi ko 38
Kothrud Bavdhan ['hmm']
Kothrud Bavdhan ['fuzzy_locality']


jambulwadi road modak vasti


palkhi chowk bhawani peth 411042
Bhawani Peth ['hmm']
Bhawani Peth ['fuzzy_ward']
Bhawani Peth ['st_ward']
Bhawani Peth ['fuzzy_locality']


sigma 1

Wanawadi Ramtekadi ['fuzzy_locality']


galli no 57 janta vasahat parvati 411009
Sinhgad Road ['hmm']
Sinhgad Road ['fuzzy_locality']


81 2 bharati nagar near puja park paud road kothrud 38
Kothrud Bavdhan ['hmm']
Kothrud Bavdhan ['fuzzy_ward']
Kothrud Bavdhan ['fuzzy_locality']


gokul nagar katraj 411046
Warje Karvenagar ['hmm']
Warje Karvenagar ['fuzzy_locality']


shri nagar dhankavdi 411046


231247 apte colony


krushnakunj apartment bajirao raod 411030
Kasba Vishrambaugwada ['hmm']
Kasba Vishrambaugwada ['fuzzy_locality']


61 14 yunaited westren society anuradha near tathavade udyan kothrud
Kothrud Bavdhan ['hmm']
Kothrud Bavdhan ['fuzzy_ward']
Kothrud Bavdhan ['fuzzy_locality']


c 0 gulankar amarbharat society s no 452 b warje
Warje Karvenagar ['hmm']
Warje Karvenagar ['fuzzy_ward']
Warje Karvenagar ['st_ward']
Warje Karvenagar ['fuzzy_locality']


103 gandhi nagar yerawada 411006
Yerawada Kalas Dhanori ['hmm']
Yerawada Kalas Dhanori ['fuzzy_ward']
Yerawada Kalas Dhanori ['f

Bibwewadi ['fuzzy_locality']


handewadi road dania apartment hadapsar 411028
Hadapsar Mundhawa ['hmm']
Hadapsar Mundhawa ['fuzzy_ward']
Hadapsar Mundhawa ['fuzzy_locality']


ofw kalpana empress hadapsar road 411013
Hadapsar Mundhawa ['hmm']
Hadapsar Mundhawa ['fuzzy_ward']
Hadapsar Mundhawa ['fuzzy_locality']


344 kasba peth
Kasba Vishrambaugwada ['hmm']
Kasba Vishrambaugwada ['fuzzy_ward']
Kasba Vishrambaugwada ['fuzzy_locality']


flat no 26 pmc bldg nagras road aundh 411007
Aundh Baner ['hmm']
Nagar Road Vadgaon Sheri ['fuzzy_ward']
Aundh Baner ['fuzzy_locality']


aundh gaon vidhate vasti aundh
Aundh Baner ['hmm']
Aundh Baner ['fuzzy_ward']
Aundh Baner ['fuzzy_locality']


shinde chawl sangam chowk shastrinagar ko
Kothrud Bavdhan ['hmm']
Kothrud Bavdhan ['fuzzy_locality']


260 7b 1 khese park lohegaon road dhanori 411015
Nagar Road Vadgaon Sheri ['hmm']
Nagar Road Vadgaon Sheri ['fuzzy_locality']


bhoi aali nanded goan
Sinhgad Road ['hmm']
Sinhgad Road ['fuzzy_locality']


jan



tadiwala road vishwajit tarun mandal
Dhole Patil Road ['hmm']
Dhole Patil Road ['fuzzy_locality']


sagar colony sr no 82 22 shastri nagar kothrud
Kothrud Bavdhan ['hmm']
Kothrud Bavdhan ['fuzzy_ward']
Kothrud Bavdhan ['fuzzy_locality']


129 shukrawar peth 411002
Kasba Vishrambaugwada ['hmm']
Kasba Vishrambaugwada ['fuzzy_locality']


kondhwa rode
Wanawadi Ramtekadi ['hmm']
Kondhawa Yewalewadi ['fuzzy_ward']
Wanawadi Ramtekadi ['fuzzy_locality']


bhavani peth timber market
Bhawani Peth ['hmm']
Bhawani Peth ['fuzzy_ward']
Bhawani Peth ['fuzzy_locality']


ramoshiwadi


gokul nagar near sai temple shevanta park dhanori 411015
Warje Karvenagar ['hmm']
Warje Karvenagar ['fuzzy_locality']


near ram mandir sagm building ambegoan pathar


e 206 meenakshi puram vadgaon bk 41
Sinhgad Road ['hmm']
Sinhgad Road ['fuzzy_locality']


ofw balaji nagar ghorpadi 411001
Kondhawa Yewalewadi ['hmm']
Kondhawa Yewalewadi ['fuzzy_locality']


618 ghorpade peth khadakmal ali 411042
Bhawani Peth ['hmm']


...

Sinhgad Road ['hmm']
Sinhgad Road ['fuzzy_locality']


janata vasahat anand math parvati payatha 411009
Sinhgad Road ['hmm']
Sinhgad Road ['fuzzy_locality']


dhan laxmi apartment shahu colony karve nagar
Warje Karvenagar ['hmm']
Warje Karvenagar ['fuzzy_prabhag']
Warje Karvenagar ['fuzzy_locality']


bombay sapras colony flat no 06 chandan nagar 411014
Nagar Road Vadgaon Sheri ['hmm']
Nagar Road Vadgaon Sheri ['fuzzy_locality']


46 b aundh road khadki 411003
Aundh Baner ['hmm']
Aundh Baner ['fuzzy_ward']
Aundh Baner ['st_ward']
Aundh Baner ['fuzzy_locality']


ofw saswad road bhekarainagar 412308


s no 648 7


paudgaon tal mulshi dharavali


405 410 s b road near j w marriot shivajijnagar 16
Shivajinagar Ghole Road ['hmm']
Shivajinagar Ghole Road ['fuzzy_ward']


gard wakadewadi old mumbai road 5
Shivajinagar Ghole Road ['hmm']
Shivajinagar Ghole Road ['fuzzy_locality']


flat no 304 laxmi nagar dhanori 411015
Yerawada Kalas Dhanori ['hmm']
Yerawada Kalas Dhanori ['fuzzy_locality']


Kothrud Bavdhan ['fuzzy_ward']
Kothrud Bavdhan ['st_ward']
Warje Karvenagar ['fuzzy_locality']


flat n 301 d 3 ganga constella kharadi
Nagar Road Vadgaon Sheri ['hmm']
Nagar Road Vadgaon Sheri ['fuzzy_locality']


48 2 6 classic kuba kondhwa 411048
Kondhawa Yewalewadi ['hmm']
Kondhawa Yewalewadi ['fuzzy_ward']
Wanawadi Ramtekadi ['fuzzy_locality']


at post shindewadi ta bhor zilla


railway gate no 20 bopodi 411020
Aundh Baner ['hmm']
Aundh Baner ['fuzzy_locality']


1449 sadashiv peth 411030
Kasba Vishrambaugwada ['hmm']
Kasba Vishrambaugwada ['fuzzy_locality']


padmavati co opp hou society balajinagar 411009
Dhankawadi Sahakarnagar ['hmm']
Dhankawadi Sahakarnagar ['fuzzy_locality']


1710 sadashiv peth 411030
Kasba Vishrambaugwada ['hmm']
Kasba Vishrambaugwada ['fuzzy_locality']


home no 788 venkatesh sadan b 10 2 rajmudra society dhankawadi 411043
Dhankawadi Sahakarnagar ['hmm']
Dhankawadi Sahakarnagar ['fuzzy_ward']
Dhankawadi Sahakarnagar ['fuzzy_locality']


indira nagar gult

Kothrud Bavdhan ['hmm']
Kothrud Bavdhan ['fuzzy_ward']


swargate police station
Kasba Vishrambaugwada ['hmm']
Kasba Vishrambaugwada ['fuzzy_locality']


laxmikant society 403 shastri nagar kothrud 411038
Kothrud Bavdhan ['hmm']
Kothrud Bavdhan ['fuzzy_ward']
Kothrud Bavdhan ['st_ward']
Kothrud Bavdhan ['fuzzy_locality']


ofw tarawade wasti mohammadwadi 411060
Hadapsar Mundhawa ['hmm']
Hadapsar Mundhawa ['st_ward']


sai nagar society kalyani nagar 411006
Nagar Road Vadgaon Sheri ['hmm']
Nagar Road Vadgaon Sheri ['fuzzy_locality']


ofw dreams wisteria undri pisoli 411060


pr 27 maruti ali samaj mandir kondhawa 411048
Kondhawa Yewalewadi ['hmm']
Kondhawa Yewalewadi ['fuzzy_ward']


1405 kasba peth 411011
Kasba Vishrambaugwada ['hmm']
Kasba Vishrambaugwada ['fuzzy_ward']
Kasba Vishrambaugwada ['st_ward']
Kasba Vishrambaugwada ['fuzzy_locality']


yewalewadi kondhwa
Kondhawa Yewalewadi ['hmm']
Kondhawa Yewalewadi ['fuzzy_ward']
Kondhawa Yewalewadi ['st_ward']
Kondhawa Yewalewadi ['fuzz

Kothrud Bavdhan ['hmm']
Kothrud Bavdhan ['fuzzy_locality']


yashashri society flat no 11 kothrud mit college road ko
Kothrud Bavdhan ['hmm']
Kothrud Bavdhan ['fuzzy_ward']
Kothrud Bavdhan ['fuzzy_locality']


pranayraj society gokulnagar dhanori road
Yerawada Kalas Dhanori ['hmm']
Yerawada Kalas Dhanori ['fuzzy_locality']


mahalaxmi palace ghar no 203 near aryan pump narhe
Sinhgad Road ['hmm']
Sinhgad Road ['fuzzy_locality']


c 302 siroco grand near punavale tanaji nagar pimpari chinchwad 411033


flat no 201 galexi park smart city infront campus school suncity rd
Sinhgad Road ['hmm']
Sinhgad Road ['fuzzy_locality']


s no 45 amarbharat society near dr rathod clinic warje
Warje Karvenagar ['hmm']
Warje Karvenagar ['fuzzy_ward']
Warje Karvenagar ['st_ward']
Warje Karvenagar ['fuzzy_locality']


balajinagar sutar galli dhankawadi 411043
Kondhawa Yewalewadi ['hmm']
Dhankawadi Sahakarnagar ['fuzzy_ward']
Kondhawa Yewalewadi ['fuzzy_locality']


kshirsagar niwas lane no 3 near mai manges

1 shri vardhamn hotel kaveri sea food warje
Warje Karvenagar ['hmm']
Warje Karvenagar ['fuzzy_ward']
Warje Karvenagar ['fuzzy_locality']


44 masjid gully yerawada 411006
Yerawada Kalas Dhanori ['hmm']
Yerawada Kalas Dhanori ['fuzzy_ward']
Yerawada Kalas Dhanori ['fuzzy_prabhag']
Yerawada Kalas Dhanori ['fuzzy_locality']


ayush neuropathy college yewalewadi
Kondhawa Yewalewadi ['hmm']
Kondhawa Yewalewadi ['fuzzy_ward']
Kondhawa Yewalewadi ['st_ward']
Kondhawa Yewalewadi ['fuzzy_locality']


sr no 20 2 near pdc bank kashinath patil nagar balaji nagar 411043
Kondhawa Yewalewadi ['hmm']
Kondhawa Yewalewadi ['fuzzy_locality']


vidya nagar near post office tingare nagar 411015


flat no 303 om sai palace narhe goan
Sinhgad Road ['hmm']
Sinhgad Road ['fuzzy_locality']


ghar dangmali ali sr no229 hadapsar
Hadapsar Mundhawa ['hmm']
Hadapsar Mundhawa ['fuzzy_ward']
Hadapsar Mundhawa ['st_ward']
Hadapsar Mundhawa ['fuzzy_locality']


tadiwala road poonam bakery
Dhole Patil Road ['hmm']
Dhole 

In [19]:
labelled_data_ward = [] # ward labelling
labelled_data_prabhag = [] # ward and prabhag labelling
labelled_data_locality = []
labelled_data_st = []
labelled_data_all = [] # all labels

for doc in docs:
    if len(doc.spans[model_label]) > 0:
        labelled_data_all.append(f"{doc.text},{wards.index(doc.spans[model_label][0].label_)}")
        if len(doc.spans['fuzzy_ward']) > 0:
            labelled_data_ward.append(f"{doc.text},{wards.index(doc.spans['fuzzy_ward'][0].label_)}")
        if len(doc.spans['fuzzy_prabhag']) > 0:
            labelled_data_prabhag.append(f"{doc.text},{wards.index(doc.spans['fuzzy_prabhag'][0].label_)}")
        if len(doc.spans['fuzzy_locality']) > 0:
            labelled_data_locality.append(f"{doc.text},{wards.index(doc.spans['fuzzy_locality'][0].label_)}")
        if len(doc.spans['st_ward']) > 0:
            labelled_data_st.append(f"{doc.text},{wards.index(doc.spans['st_ward'][0].label_)}")
                        
            

In [20]:
# generating files for clustering (considering only the addresses named by wards and prabhags)
model_data = list(set(labelled_data_ward + labelled_data_prabhag))

random.shuffle(model_data)

# splitting 80-20 for training and testing
train_data = model_data[:int(len(model_data)/1.25)]
eval_data = model_data[int(len(model_data)/1.25):]

with open('train.csv', 'w') as f:
    for i in train_data:
        f.write(i+'\n')

with open('eval.csv', 'w') as f:
    for i in eval_data:
        f.write(i+'\n')

In [21]:
labelled_data_all

['c2 1104 bel castle society lonkar mala keshav nagar mundhwa,0',
 'koregaon park lane no 5 nr devi mandir kawdewadi,9',
 'janta vasahat parvati paytha 124 infront pmc school 09,11',
 'kale padal hadapsar railway crossing,0',
 's n 53 1 sanathnagar kharadi,12',
 'g 16 sai ganesh complex karvenagar near icici bank,3',
 'sahayog nagar s no 45 warje,3',
 'sunfurnasi jambulwadi road e 401 dattanagar katraj,8',
 'lane 9 flat n 313 tulajabhavani nagar kharadi,12',
 'hingane hom colony kale farm near ganpati mandir karenagar,3',
 'karvenagar lane no 7 hou no 54 canol road,3',
 'sadguru plat dangat patil nagar shivane punr,11',
 'kalapataru colony swapnil smruti warje near jakat naka warje,3',
 'chandan nagar kharadi,12',
 'pr 24 ramtekadi hadapsar 411013,0',
 'pr 24 ramnagar hadapsar near andh school 411013,7',
 'panchami p g rakshaknagar kharadi,12',
 'b 29 05 indiranagar lower bibwewadi 411037,4',
 'patra chawl sawantnagari bopodi,9',
 '5 59 lokmanya nagar navi peth 411030,1',
 'shankar tea

In [23]:
print(f"Total addresses: {len(corrected_addr)}")
print(f"Total labelled addresses: {len(labelled_data_all)}")
print(f"Labelled addresses by Wards: {len(labelled_data_ward)}")
print(f"Labelled addresses by Prabhags: {len(labelled_data_prabhag)}")
print(f"Labelled addresses by Localities: {len(labelled_data_locality)}")
print(f"Labelled addresses by similarities in sentence transformers: {len(labelled_data_st)}")

Total addresses: 6322
Total labelled addresses: 4616
Labelled addresses by Wards: 2090
Labelled addresses by Prabhags: 517
Labelled addresses by Localities: 4472
Labelled addresses by similarities in sentence transformers: 616
